1. 采样
2. 混合精度训练
3. 持续预训练

In [ ]:
! pip install --upgrade torch==1.6.0

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 748.8MB 8.4MB/s eta 0:00:013    |███▍                            | 79.1MB 10.1MB/s eta 0:01:07     |█████████████▉                  | 323.1MB 11.6MB/s eta 0:00:37     |█████████████████▎              | 404.9MB 11.2MB/s eta 0:00:31     |██████████████████████████▏     | 611.6MB 9.7MB/s eta 0:00:15     |████████████████████████████▎   | 661.1MB 10.6MB/s eta 0:00:09     |███████████████████████████████ | 727.0MB 7.8MB/s eta 0:00:03
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0


In [ ]:
! pip install torchvision==0.7.0

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 5.9MB 17.2MB/s eta 0:00:01
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0


In [ ]:
import torch
import os
import random
import numpy as np
config = {
        'train_file_path': 'data/data100821/train.json',
        'dev_file_path': 'data/data100821/dev.json',
        'test_file_path': 'data/data100821/test.json',
        'output_path': '.',
        'model_path': 'data/data94445',
        'batch_size': 64,
        'num_epochs': 1,
        'max_seq_len': 64,
        'learning_rate': 2e-5,
        'eps': 0.1,
        'alpha': 0.3,
        'adv': 'fgm',
        'warmup_ratio': 0.05,
        'weight_decay': 0.01,
        'use_bucket': True,
        'bucket_multiplier': 200,
        'device': 'cuda',
        'n_gpus': 0,
        'use_amp': True,  # 只针对有 tensor core 的gpu有效
        'logging_step': 400,
        'ema_start_step': 500,
        'ema_start': False,
        'seed': 2021
    }

if not torch.cuda.is_available():
    config['device'] = 'cpu'
else:
    config['n_gpus'] = torch.cuda.device_count()
    config['batch_size'] *= config['n_gpus']

if not os.path.exists(config['output_path']):
    os.makedirs((config['output_path']))

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2021

In [ ]:
! pip install transformers==4.0.1

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 1.4MB 12.8MB/s eta 0:00:01
     |████████████████████████████████| 40kB 37.9MB/s eta 0:00:01
     |████████████████████████████████| 2.9MB 32.2MB/s eta 0:00:01


In [ ]:
from tqdm import tqdm
import json
import pandas as pd
def parse_data(path, data_type='train'):
    sentence_a = []
    sentence_b = []
    labels = []
    with open(path, 'r', encoding='utf8') as f:
        for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
            line = json.loads(line)
            sentence_a.append(line['sentence1'])
            sentence_b.append(line['sentence2'])
            if data_type != 'test':
                labels.append(int(line['label']))
            else:
                labels.append(0)
    df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns=['text_a', 'text_b', 'labels'])
    return df

In [ ]:
# inputs defaultdict(list)
def build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
    # add_special_tokens [CLS] [SEP]
    # return_token_type_ids 该词属于sentence_a or sentence_b. 
    # return_attention_mask pad=0, 不是pad的部分标为1， 是pad标为0. 
    inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
    inputs['input_ids'].append(inputs_dict['input_ids'])
    inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
    inputs['attention_mask'].append(inputs_dict['attention_mask'])
    inputs['labels'].append(label)

In [ ]:
from collections import defaultdict
def read_data(config, tokenizer):
    train_df = parse_data(config['train_file_path'], data_type='train')
    dev_df = parse_data(config['dev_file_path'], data_type='dev')
    test_df = parse_data(config['test_file_path'], data_type='test')

    data_df = {'train': train_df, 'dev': dev_df, 'test': test_df}
    # 保存BERT的输入
    processed_data ={}

    for data_type, df in data_df.items():
        inputs = defaultdict(list)
        for i, row in tqdm(df.iterrows(), desc=f'Preprocessing {data_type} data', total=len(df)):
            label = row[2]
            sentence_a, sentence_b = row[0], row[1]
            build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer)
        
        processed_data[data_type] = inputs
    
    return processed_data

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config['model_path'])
dt = read_data(config, tokenizer)

Preprocessing test data: 100%|██████████| 3861/3861 [00:01<00:00, 2081.18it/s]


In [ ]:
print(dt['train']['input_ids'][0])

[101, 6010, 6009, 955, 1446, 5023, 7583, 6820, 3621, 1377, 809, 2940, 2768, 1044, 2622, 1400, 3315, 1408, 102, 955, 1446, 3300, 1044, 2622, 1168, 3309, 6820, 3315, 1408, 102]


In [ ]:
from torch.utils.data import Dataset
class AFQMCDataset(Dataset):
    def __init__(self, data_dict):
        super(AFQMCDataset, self).__init__()
        self.data_dict = data_dict
    
    def __getitem__(self, index):
        data = (self.data_dict['input_ids'][index], self.data_dict['token_type_ids'][index],
                self.data_dict['attention_mask'][index], self.data_dict['labels'][index])
        return data
    
    def __len__(self):
        return len(self.data_dict['input_ids'])

In [ ]:
class Collator:
    def __init__(self, max_seq_len, tokenizer):
        self.max_seq_len = max_seq_len
        self.tokenizer = tokenizer
    
    def pad_and_truncate(self, input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len):
        input_ids = torch.zeros((len(input_ids_list), max_seq_len), dtype=torch.long)
        token_type_ids = torch.zeros_like(input_ids)
        attention_mask = torch.zeros_like(input_ids)

        for i in range(len(input_ids_list)):
            seq_len = len(input_ids_list[i])

            if seq_len <=max_seq_len:
                input_ids[i, :seq_len] = torch.tensor(input_ids_list[i], dtype=torch.long)
                token_type_ids[i, :seq_len] = torch.tensor(token_type_ids_list[i], dtype=torch.long)
                attention_mask[i, :seq_len] = torch.tensor(attention_mask_list[i], dtype=torch.long)
            
            else:
                # input_ids 最后一位放上一个特殊的token
                input_ids[i] = torch.tensor(input_ids_list[i][:max_seq_len-1] + [self.tokenizer.sep_token_id], 
                                            dtype=torch.long)
                # token_type_ids[i] = torch.tensor(token_type_ids_list[i][:max_seq_len-1] + [self.tokenizer.sep_token_id], 
                #                             dtype=torch.long)
                # attention_mask[i] = torch.tensor(attention_mask_list[i][:max_seq_len-1] + [self.tokenizer.sep_token_id], 
                #                             dtype=torch.long)
                token_type_ids[i] = torch.tensor(token_type_ids_list[i][:max_seq_len], dtype=torch.long)
                attention_mask[i] = torch.tensor(attention_mask_list[i][:max_seq_len], dtype=torch.long)
        
        labels = torch.tensor(labels_list, dtype=torch.long)
        return input_ids, token_type_ids, attention_mask, labels


    def __call__(self, examples):
        input_ids_list, token_type_ids_list, attention_mask_list, labels_list = list(zip(*examples))
        cur_max_seq_len = max(len(input_id) for input_id in input_ids_list)
        max_seq_len = min(cur_max_seq_len, self.max_seq_len)

        input_ids, token_type_ids, attention_mask, labels = self.pad_and_truncate(input_ids_list, 
                                                                                  token_type_ids_list, attention_mask_list,
                                                                                  labels_list, max_seq_len)
        data_dict = {
            'input_ids': input_ids,
            'token_type_ids': token_type_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }
        return data_dict


In [ ]:
collate_fn = Collator(config['max_seq_len'], tokenizer)

# 采样

![build_dataloader](https://img-blog.csdnimg.cn/ea1002a506ee4eb78e9cfb1bc4c0595c.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![Dataloader](https://img-blog.csdnimg.cn/b80cee8a1c7d49b79e7b80cc81150d66.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

# 所有采样器都继承自Sampler这个类
# 每个Sampler子类都要实现__iter__方法【迭代数据集example索引的方法】，以及返回迭代器长度的__len__方法

![sampler](https://img-blog.csdnimg.cn/1c40aedade9f40a493b4df97d0c1def0.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![sequentialsampler](https://img-blog.csdnimg.cn/9e8ee018cea84729ac6b5742395d8ea2.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

在初始化时拿到数据集data_source， 按顺序对元素进行采样，每次只返回一个索引值

In [ ]:
import torch
# (2,3,4) batch_size:2 seq_len=3, embedding_dim=4，每个 batch 有2条数据，每个句子包含3个词， 每个词的维度是4
a = torch.randperm(24).reshape((2,3,4))
print(a)
b = torch.utils.data.SequentialSampler(a)
print(b)
for i in b:
    print(i)

tensor([[[14,  3, 15,  5],
         [ 2, 20,  9, 16],
         [19, 21,  7, 17]],

        [[22, 12, 23,  8],
         [ 6, 10, 18,  1],
         [ 4, 13, 11,  0]]])
0
1


## replacement : True 表示可以重复采样
# num_samples:  指定采样的数量
PS:当使用replacement=False，不应制定num_samples

![randomsampler](https://img-blog.csdnimg.cn/9d2e2afdbe4d4df4aee3102e46054650.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [ ]:
a = torch.randperm(60).reshape((5,3,4))
print(a)
b = torch.utils.data.RandomSampler(a, replacement=True, num_samples=3)
print(b)
for i in b:
    print(i)

tensor([[[54, 49, 14, 52],
         [19, 34, 26, 29],
         [15, 45, 43, 59]],

        [[56, 27, 44, 22],
         [25, 38, 33, 51],
         [30, 47, 10,  9]],

        [[ 1, 53,  7,  0],
         [50, 28, 39,  6],
         [11, 17, 58, 48]],

        [[ 2, 46,  5, 12],
         [24, 36,  4, 16],
         [ 3, 31, 41, 18]],

        [[23, 40, 35,  8],
         [32, 37, 20, 13],
         [55, 42, 57, 21]]])
1
1
3


SubsetRandomSampler： 从给定的索引列表中随机采样元素，不放回采样
indices(sequence): 索引序列

![sunsetRandomSampler](https://img-blog.csdnimg.cn/e80f6a1bafe042f28da652dc5a2388ab.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [ ]:
a = torch.randperm(60).reshape((5,3,4))
print(a)

tensor([[[13,  8, 22, 25],
         [37, 39,  6, 20],
         [27, 35, 53, 26]],

        [[33, 40, 19, 10],
         [30, 17,  0, 55],
         [18, 29,  2,  5]],

        [[12, 34, 32, 16],
         [ 9, 57, 48, 23],
         [14,  4, 59, 41]],

        [[11, 24, 28, 44],
         [ 7, 51, 15,  1],
         [58, 31, 21, 54]],

        [[36, 52, 42, 38],
         [45, 43, 56, 49],
         [ 3, 46, 47, 50]]])


In [ ]:
b = torch.utils.data.SubsetRandomSampler(indices=a[2:])
for i in b:
    print(i)

tensor([[12, 34, 32, 16],
        [ 9, 57, 48, 23],
        [14,  4, 59, 41]])
tensor([[36, 52, 42, 38],
        [45, 43, 56, 49],
        [ 3, 46, 47, 50]])
tensor([[11, 24, 28, 44],
        [ 7, 51, 15,  1],
        [58, 31, 21, 54]])


sampler: 基采样器
batch_size: 
drop_last=True, 如果一个batch的长度小于batch_size则丢弃

![BatchSampler](https://img-blog.csdnimg.cn/8a1b2f5ae320453c9fae8ae8e0ef2080.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [ ]:
a = torch.randperm(60).reshape((5,3,4))
# print(a)
b = torch.utils.data.BatchSampler(torch.utils.data.RandomSampler(a), 2, drop_last=True)
for i in b:
    print(i)

[2, 3]
[0, 1]


![BucketSampler](https://img-blog.csdnimg.cn/6413cea5dfbf4494a6b2b64504f74a97.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![SortedSampler](https://img-blog.csdnimg.cn/64f60217df474cf1b0d7aa1c3558cc1f.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![BucketSampler](https://img-blog.csdnimg.cn/d7e03938f2824f9cb8a6c3a895f5a78a.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [ ]:
mini_dataset = {k: v[:6] for k, v in dt['train'].items()}
mini_data = AFQMCDataset(mini_dataset)
print(mini_data)
# mini_data 的前6条数据的长度
for i, d in enumerate(mini_data):
    print(len(d[0]))

30
24
25
24
59
28


In [ ]:
from extra_file.bucket_sampler import SortedSampler
random_sampler = torch.utils.data.RandomSampler(mini_data, replacement=False)
print(list(random_sampler))
# [3, 5, 4, 1, 0, 2]

batch_sampler = torch.utils.data.BatchSampler(random_sampler, 4, drop_last=True)
# [0, 5, 2, 4]

for samp in batch_sampler:
    print(samp)
    sorted_sampler = SortedSampler(samp, sort_key=lambda x:len(mini_data[x][0]))
    print(list(sorted_sampler))

[3, 5, 4, 1, 0, 2]
[0, 5, 2, 4]
[2, 1, 0, 3]


[2, 1, 0, 3] 
2 -> 2 -> 25
1 -> 5 -> 28
0 -> 0 -> 30
3 -> 4 -> 59

In [ ]:
c = list(torch.utils.data.BatchSampler(sorted_sampler, 2, drop_last=True))
print(c)

[[2, 1], [0, 3]]


In [ ]:
for batch in torch.utils.data.SubsetRandomSampler(c):
    print('从给定的索引列表中随机采样元素')
    print(batch)
    print('所对应的原序列是什么：')
    print([samp[i] for i in batch])

从给定的索引列表中随机采样元素
[2, 1]
所对应的原序列是什么：
[2, 5]
从给定的索引列表中随机采样元素
[0, 3]
所对应的原序列是什么：
[0, 4]


假设：batch_size = 2， 经常使用的  返回 一个batch(包含两条数据)
桶采样： DataSet -> 搞出一个大桶（n * batch_size） -> 对于 n * 2 条数据进行排序 -> 从中抽取 batch_size（2）条（小桶）数据

In [ ]:
from time import sleep
from tqdm import tqdm
for i in tqdm(range(60*5), desc='现在是休息时间，看录播的同学可以跳过哦～'):
    sleep(1)

现在是休息时间，看录播的同学可以跳过哦～:  10%|█         | 30/300 [00:30<04:30,  1.00s/it]

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler
from extra_file.bucket_sampler import BucketBatchSampler

def build_dataloader(config, data, collate_fn):
    train_dataset = AFQMCDataset(data['train'])
    dev_dataset = AFQMCDataset(data['dev'])
    test_dataset = AFQMCDataset(data['test'])

    if config['use_bucket']:
        train_sampler = RandomSampler(train_dataset)
        bucket_sampler = BucketBatchSampler(train_sampler, batch_size=config['batch_size'],
                                            drop_last=False, sort_key=lambda x:len(train_dataset[x][0]),
                                            bucket_size_multiplier=config['bucket_multiplier'])
        train_dataloder = DataLoader(dataset=train_dataset, batch_sampler=bucket_sampler, 
                                    num_workers=4, collate_fn=collate_fn)
    
    else:
        train_dataloder = DataLoader(train_dataset, batch_size=config['batch_size'], 
                                     shuffle=True, num_workers=4, collate_fn=collate_fn)
    
    dev_dataloader = DataLoader(dev_dataset, batch_size=config['batch_size'], 
                                     shuffle=False, num_workers=4, collate_fn=collate_fn)
    
    test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], 
                                     shuffle=False, num_workers=4, collate_fn=collate_fn)
    
    return train_dataloder, dev_dataloader, test_dataloader

In [ ]:
train_dataloder, dev_dataloader, test_dataloader = build_dataloader(config, dt, collate_fn)

In [ ]:
for i in train_dataloder:
    print(i)
    break

{'input_ids': tensor([[ 101, 2769, 4638,  ..., 5709, 1446,  102],
        [ 101, 2769, 4638,  ..., 3119, 7178,  102],
        [ 101, 1963,  862,  ..., 1525, 7027,  102],
        ...,
        [ 101, 3118,  802,  ...,  679, 6629,  102],
        [ 101, 2972, 2408,  ...,  802, 1408,  102],
        [ 101,  955, 1446,  ..., 6820, 3621,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
        1, 0, 0

# 自动混合精度

![amp](https://img-blog.csdnimg.cn/e4226734b82f462e983aa905de50891a.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

# 混合精度训练
作用：训练时，尽量不降低性能，并提升速度
Float16优点:
* 减少内存的使用
* 加快训练和推断的计算，能带来多一倍速的体验

Float16缺点:
* 溢出错误
* 舍入误差

In [ ]:
a = torch.zeros(2,3)
print(a.type())

torch.FloatTensor


![amp2](https://img-blog.csdnimg.cn/72b642b508024cc2a6207c308347c7e7.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

当进入autocast()时， 系统自动切换为float16, autocast上下文只包含前向传播，建议不用反向传播

![scaling](https://img-blog.csdnimg.cn/bbae5cdd360748ecb59cee8dc6f728f2.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

* scaler.scale(loss) 将给定的损失成一缩放器的当前比例因子，进行反向传播
* scaler.step(optimizer) 取消缩放梯度并调用optimizer.step()
* scaler.update() 更新缩放器的比例因子

![GradScaler](https://img-blog.csdnimg.cn/2c0fdf08602748ea8a7655f2d5bb1829.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![example](https://img-blog.csdnimg.cn/dfeebde4d34b496096062bb7dbbee7b6.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [ ]:
def evaluation(config, model, val_dataloader):
    model.eval()
    preds = []
    labels = []
    val_loss = 0.
    val_iterator = tqdm(val_dataloader, desc='Evaluation', total=len(val_dataloader))

    with torch.no_grad():
        for batch in val_iterator:
            labels.append(batch['labels'])
            batch_cuda = {item: value.to(config['device']) for item, value in list(batch.items())}
            loss, logits = model(**batch_cuda)[:2]

            if config['n_gpus'] > 1:
                loss = loss.mean()

            val_loss += loss.item()
            preds.append(logits.argmax(dim=-1).detach().cpu())

    avg_val_loss = val_loss / len(val_dataloader)
    labels = torch.cat(labels, dim=0).numpy()
    preds = torch.cat(preds, dim=0).numpy()
    f1 = metrics.f1_score(labels, preds)
    acc = metrics.accuracy_score(labels, preds)
    return avg_val_loss, f1, acc

In [ ]:
class EMA:
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}
        self.register()

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

对抗样本：对人类看起来一样，对模型来说预测结果却完全不一样的样本。

1.相对于原始输入（词向量），所添加的扰动是微小的 2.模型犯错

对抗训练：在训练集中增加一些对抗样本，希望增强模型对于对抗样本的鲁棒性

![对抗训练](https://img-blog.csdnimg.cn/20210716113626200.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

## FGM
对于每个x:
1. 计算x的前向loss, 反向传播得到梯度；
2. 根据embeddign矩阵计算的梯度计算出r, 并加到当前embedding上，相当于x+r
3. 计算x+r的前向loss, 反向传播得到梯度，然后累加到(1)的梯度上；
4. 将embedding恢复为（1）时的embedding；
5. 根据（3）的梯度对参数进行更新。

## PGD
FGM是一下子算出了对抗扰动，这样得到的扰动不一定是最优的。因此PGD进行了改进，多迭代了K/t次，慢慢找到最优的扰动
对于每个x:
1. 计算x的前向loss, 反向传播得到梯度；
  对于每步t：
  2. 根据embeddign矩阵计算的梯度计算出r, 并加到当前embedding上，相当于x+r；
  3. t如果不是最后一步，将梯度归0， 根据2的x+r计算前后向并得到梯度
  4. t是最后一步，恢复1的梯度，计算最后的x+r并将梯度累加到(1)上
5. 将embedding恢复为（1）时的embedding；
6.根据（4）的梯度对参数进行更新。

In [ ]:
from transformers import BertForSequenceClassification
from torch.cuda import amp
from transformers import AdamW
from extra_file.extra_pgd import *
from extra_file.extra_loss import *
from extra_file.extra_fgm import *
from extra_file.extra_optim import *
from tqdm import trange
def train(config, train_dataloader, dev_dataloader):
    # 封装好 BertForSequenceClassification
    model = BertForSequenceClassification.from_pretrained(config['model_path'])
    

    # param_optimizer = model.named_parameters()
    param_optimizer = list(model.named_parameters())


    # 实例化scaler对象 enabled=True 可以使用梯度缩放
    scaler = amp.GradScaler(enabled=config['use_amp'])
    
    # 权重缩减
    no_decay = ['bias', 'LayerNorm.weight']
    
    # 名称包含 ['bias', 'LayerNorm.weight']的权重， 其权重衰减因子为0
    # 名称不包含 ['bias', 'LayerNorm.weight']的权重， 其权重衰减因子为 0.01
    optimizer_grouped_parameters = [
        {"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         "weight_decay": config['weight_decay']},
        {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         "weight_decay": 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=config['learning_rate'], eps=1e-8)
    
    # lookahead 预先查看由 AdamW 生成的快速权重 来选择搜索方向
    optimizer = Lookahead(optimizer, 5, 1)
    total_steps = config['num_epochs'] * len(train_dataloader)

    # 使用Warmup来调整学习率，每调用warmup_steps次，对应的学习率就会调整一次。
    lr_scheduler = WarmupLinearSchedule(optimizer,
                                        warmup_steps=int(config['warmup_ratio'] * total_steps),
                                        t_total=total_steps)
    model.to(config['device'])

    if config['adv'] == 'fgm':
        fgm = FGM(model)
    else:
        pgd = PGD(model)
        K = 3
    
    epoch_iterator = trange(config['num_epochs'])
    global_steps = 0
    train_loss = 0.
    logging_loss = 0.
    best_acc = 0.
    best_model_path = ''
    
    # 多卡 情况
    if config['n_gpus'] > 1:
        model = nn.DataParallel(model)

    for _ in epoch_iterator:

        train_iterator = tqdm(train_dataloader, desc='Training', total=len(train_dataloader))
        model.train()
        for batch in train_iterator:
            batch_cuda = {item: value.to(config['device']) for item, value in list(batch.items())}
            
            # 前向过程（前向传播 + loss）
            with amp.autocast(enabled=config['use_amp']):
                loss = model(**batch_cuda)[0]
                # 多卡 取平均
                if config['n_gpus'] > 1:
                    loss = loss.mean()
            

            scaler.scale(loss).backward()

            if config['adv'] == 'fgm':
                # 在embedding上加扰动
                fgm.attack(epsilon=config['eps'])
                
                # autocast
                with amp.autocast(enabled=config['use_amp']):
                    loss_adv = model(**batch_cuda)[0]
                   
                    if config['n_gpus'] > 1:
                        loss_adv = loss_adv.mean()
                
                scaler.scale(loss_adv).backward()
                # 恢复embedding参数
                fgm.restore()
            else:
                pgd.backup_grad()
                for t in range(K):
                    pgd.attack(epsilon=config['eps'], alpha=config['alpha'], is_first_attack=(t == 0))
                    if t != K - 1:
                        model.zero_grad()
                    else:
                        pgd.restore_grad()
                    with amp.autocast(enabled=config['use_amp']):
                        loss_adv = model(**batch_cuda)[0]
                        if config['n_gpus'] > 1:
                            loss_adv = loss_adv.mean()

                    scaler.scale(loss_adv).backward()
                pgd.restore()

            scaler.step(optimizer)
            scaler.update()

            lr_scheduler.step()
            optimizer.zero_grad()

            if config['ema_start']:
                ema.update()

            train_loss += loss.item()
            global_steps += 1

            train_iterator.set_postfix_str(f'running training loss: {loss.item():.4f}')

            if global_steps % config['logging_step'] == 0:
                if global_steps >= config['ema_start_step'] and not config['ema_start']:
                    print('\n>>> EMA starting ...')
                    config['ema_start'] = True
                    
                    ema = EMA(model.module if hasattr(model, 'module') else model, decay=0.999)

                print_train_loss = (train_loss - logging_loss) / config['logging_step']
                logging_loss = train_loss


                if config['ema_start']:
                    ema.apply_shadow()
                val_loss, f1, acc = evaluation(config, model, dev_dataloader)

                print_log = f'\n>>> training loss: {print_train_loss:.6f}, valid loss: {val_loss:.6f}, '

                if acc > best_acc:
                    model_save_path = os.path.join(config['output_path'],
                                                   f'checkpoint-{global_steps}-{acc:.6f}')
                    model_to_save = model.module if hasattr(model, 'module') else model
                    model_to_save.save_pretrained(model_save_path)
                    best_acc = acc
                    best_model_path = model_save_path

                print_log += f'valid f1: {f1:.6f}, valid acc: {acc:.6f}'

                print(print_log)
                model.train()

                if config['ema_start']:
                    ema.restore()

    return model, best_model_path

In [54]:
train(config, train_dataloder, dev_dataloader)

Some weights of the model checkpoint at data/data94445 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at da

KeyboardInterrupt: 